# Project: Predicting Stock Price & Return  
Extrat Transform Load (ETL)
## ver 0.1  
All guidance and guildlines provided by Romeo Kienzler

### Guidance - Extract, transform, load (ETL)  
This task is an important step in transforming the data from the source system into data suitable for analytics. In traditional data warehousing, this process includes accessing the online transaction processing (OLTP) system’s databases, transforming the data from a highly normalized data model into a Star or Snowflake Schema, and storing the data to a data warehouse. In data science projects, this step is usually much simpler. The data arrives in an exported format (for example, JSON or CSV). But, sometimes de-normalization must be done as well. The result usually ends up in a bulk storage like Cloud Object Store.

### Guildlines for Data Cleansing
During the transformation, we will also perform data cleansing. Here are the Guildlinesfrom IBM cloud garage method:  
- Data types  
Are data types of columns matching their content? E.g. is age stored as integer and not as string?
- Ranges  
Does the value distribution of values in a column make sense? Use stats (e.g. min, max, mean, standard deviation) and visualizations (e.g. boxplot, histogram) for help
- Emptiness  
Are all values non-null where mandaroty? E.g. client IDs  
- Uniqueness  
Are duplicates present where undesired? E.g. client IDs
- Set memberships  
Ar only allowed valus chosen for categorical or ordinal fields? E.g. Female, Male, Unknown
- Foreign key set memberships  
Are only allowed values chosen a field? E.g. ZIP code  
- Cross-field validation  
Some fields can impact validity of other fields. E.g. a male person can't be pregnant



### Data Integration - Technology Choice and Justification  

The data is integrated using pandas join function, because all datasets are indexed on datetimes series in a same perid, and the size of dataset is relatively small.

### Data Repository - Technology Choice and Justification  
The data is saved as csv files in the project directory. The size of dataset is relatively small, so no extra need for advanced data repository technique.

#### 0. Import packages

In [367]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [368]:
#let the notebook display full length of the data columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

#### 1. Extract Data

##### Define extract functions  
These functions can be easily changed if we want to change the data acquisition methods.

In [369]:
def extract_stock_data(name):
    '''
    read stock data from csv file
    '''
    data = pd.read_csv(name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [370]:
def extract_indecis_data(name):
    '''
    read index data from csv file
    '''
    data = pd.read_csv(name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [371]:
def extract_exchange_data(name):
    '''
    read exchange data from csv file
    '''
    data = pd.read_csv(name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [372]:
def extract_trend_data(name):
    '''
    read trend data from csv file
    '''
    data = pd.read_csv("trend_" + name + ".csv", parse_dates=['date'], index_col='date')
    return data

In [373]:
def extract_data(stock_list=['loblaw', 'metro', 'empa', 'gwl', 'atd', 'tsx', 'sp500'],
                 indecis_list=['BCPI', 'CPI', 'bank_interest'], 
                 exchange_list=['CEER'], 
                 trend_list=['grocery_store', 'loblaws', 'stock']):

    data = pd.DataFrame()
    for stock in stock_list:
        stock_data = extract_stock_data(stock)
        stock_data.columns = [stock + '_price', stock + '_volume']
        data = data.join(stock_data, how='outer')

    for index in indecis_list:
        indecis_data = extract_indecis_data(index)
        indecis_data.columns = [index]
        data = data.join(indecis_data, how='outer')

    for exchange in exchange_list:
        exchange_data = extract_indecis_data(exchange)
        exchange_data.columns = [exchange]
        data = data.join(exchange_data, how='outer')

    for trend in trend_list:
        trend_data = extract_trend_data(trend)
        trend_data.columns = ["trend_" + trend]
        data = data.join(trend_data, how='outer')

    return data


##### Extract the data using the functions  
Inspect the data and we will see lots of missing values. This is because the data have different starting and ending date, and some data have lower frequency such as weekly and monthly.

In [374]:
data_extracted = extract_data(stock_list=['loblaw', 'metro', 'empa', 'gwl', 'atd', 'tsx', 'sp500'],
                              indecis_list=['BCPI', 'CPI', 'bank_interest'], 
                              exchange_list=['CEER'], 
                              trend_list=['grocery_store', 'loblaws', 'stock'])

In [375]:
data_extracted.head()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.43,NaN,NaN,NaN
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.47,NaN,NaN,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.84,NaN,NaN,NaN
1999-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.66,NaN,NaN,NaN
1999-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.46,NaN,NaN,NaN


In [376]:
data_extracted.tail()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2019-10-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2966.1499,2.557020e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-15,72.60,446100.0,56.59,627000.0,34.92,386557.0,109.30,107100.0,39.68,1290026.0,16418.4004,212333000.0,2995.6799,3.340740e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-16,71.62,716200.0,56.06,435900.0,34.70,805510.0,109.22,170900.0,39.58,1623424.0,16427.1992,165968300.0,2989.6899,3.222570e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-17,71.50,528200.0,56.23,700600.0,35.13,576685.0,109.28,120000.0,39.17,1868090.0,16426.3008,168397700.0,2997.9500,3.115960e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-18,71.22,284800.0,55.95,633400.0,34.84,458759.0,108.84,72500.0,39.02,1669638.0,16377.0996,166347800.0,2986.2000,3.264290e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [377]:
data_extracted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5495 entries, 1999-01-01 to 2019-10-18
Data columns (total 21 columns):
loblaw_price           5019 non-null float64
loblaw_volume          5019 non-null float64
metro_price            5020 non-null float64
metro_volume           5020 non-null float64
empa_price             5020 non-null float64
empa_volume            5020 non-null float64
gwl_price              5019 non-null float64
gwl_volume             5019 non-null float64
atd_price              5020 non-null float64
atd_volume             5020 non-null float64
tsx_price              5020 non-null float64
tsx_volume             5020 non-null float64
sp500_price            4981 non-null float64
sp500_volume           4981 non-null float64
BCPI                   1030 non-null float64
CPI                    237 non-null float64
bank_interest          1030 non-null float64
CEER                   5421 non-null float64
trend_grocery_store    190 non-null float64
trend_loblaws         

In [378]:
data_extracted.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,5019.000000,5.019000e+03,5020.000000,5.020000e+03,5020.000000,5.020000e+03,5019.000000,5.019000e+03,5020.000000,5.020000e+03,5020.000000,5.020000e+03,4981.000000,4.981000e+03,1030.000000,237.000000,1030.000000,5421.000000,190.000000,190.000000,190.000000
mean,35.579635,5.293163e+05,16.246786,7.024548e+05,13.307532,2.281543e+05,70.817707,1.156871e+05,9.950892,2.032111e+06,11835.349384,1.720116e+08,1559.498333,3.087279e+09,474.326641,115.605907,4.006359,119.529555,51.421053,38.878947,26.221053
std,12.863246,5.922035e+05,15.055826,8.051032e+05,8.321394,2.974725e+05,21.306746,1.034325e+05,12.264966,2.337872e+06,2967.700788,7.576097e+07,567.903560,1.478798e+09,133.807475,11.783501,1.397609,13.244301,15.005169,8.555519,20.481081
min,0.000000,0.000000e+00,1.101700,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.244500,0.000000e+00,0.000000,0.000000e+00,676.530000,3.560700e+08,222.700000,93.500000,2.250000,95.430000,34.000000,24.000000,11.000000
25%,26.289250,2.512000e+05,4.063425,3.465000e+05,5.508300,6.057675e+04,55.708500,5.835000e+04,1.524100,9.528420e+05,9276.599600,1.238391e+08,1149.500000,1.662000e+09,365.982500,105.400000,3.000000,110.300000,41.250000,35.000000,14.000000
50%,32.522400,4.141000e+05,8.986750,5.455500e+05,11.492800,1.478535e+05,68.336900,9.200000e+04,2.747900,1.562724e+06,12262.300300,1.720943e+08,1355.810100,3.212320e+09,451.600000,115.600000,3.700000,118.330000,46.000000,38.000000,17.000000
75%,44.048600,6.444000e+05,23.034000,8.541750e+05,20.067175,3.018135e+05,87.773950,1.393000e+05,17.332400,2.369748e+06,14201.124750,2.135500e+08,1966.970000,3.919240e+09,580.987500,125.700000,4.750000,131.310000,60.000000,41.000000,26.750000
max,75.770000,1.482520e+07,58.520000,3.157200e+07,37.233700,5.316156e+06,117.956200,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,7.500000,150.480000,100.000000,100.000000,100.000000



    
#### 2. Transform the data

In [379]:
def transform_data(data_extracted,
                              stock_list=['loblaw', 'metro', 'empa', 'gwl', 'atd', 'tsx', 'sp500'],
                              indecis_list=['BCPI', 'CPI', 'bank_interest'], 
                              exchange_list=['CEER'], 
                              trend_list=['grocery_store', 'loblaws', 'stock']):
    '''
    tranform the data into a usable form. include slicing the date, interpolating values, etc.
    '''
    data_extracted = data_extracted.resample('B').interpolate()

    data_extracted = data_extracted['2003':]

    data_extracted = data_extracted[1:]

    for column in [column for column in data_extracted.columns if ('price' in column) or ('volume' in column)]:
        for row in data_extracted[data_extracted[column] == 0].iterrows():
            '''
            last_price = data_extracted.loc[:row[0]].tail(2).head(1)
            while last_price[column].squeeze() == 0:
                last_price = data_extracted.loc[:last_price.index.max()].tail(2).head(1)
            last_price = last_price[column].squeeze()

            next_price = data_extracted.loc[row[0]:].head(2).tail(1)
            while next_price[column].squeeze() == 0:
                next_price = data_extracted.loc[next_price.index.min():].head(2).tail(1)
            next_price = next_price[column].squeeze()

            data_extracted.loc[row[0]][column] = np.mean([last_price, next_price])
            '''
            # The previous solution was to use mean value to subsitute the 0 value of price, now we decide to drop these rows
            data_extracted.drop(row[0], inplace=True)

    return data_extracted


In [380]:
data_transformed = transform_data(data_extracted)

In [381]:
data_extracted.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,5019.000000,5.019000e+03,5020.000000,5.020000e+03,5020.000000,5.020000e+03,5019.000000,5.019000e+03,5020.000000,5.020000e+03,5020.000000,5.020000e+03,4981.000000,4.981000e+03,1030.000000,237.000000,1030.000000,5421.000000,190.000000,190.000000,190.000000
mean,35.579635,5.293163e+05,16.246786,7.024548e+05,13.307532,2.281543e+05,70.817707,1.156871e+05,9.950892,2.032111e+06,11835.349384,1.720116e+08,1559.498333,3.087279e+09,474.326641,115.605907,4.006359,119.529555,51.421053,38.878947,26.221053
std,12.863246,5.922035e+05,15.055826,8.051032e+05,8.321394,2.974725e+05,21.306746,1.034325e+05,12.264966,2.337872e+06,2967.700788,7.576097e+07,567.903560,1.478798e+09,133.807475,11.783501,1.397609,13.244301,15.005169,8.555519,20.481081
min,0.000000,0.000000e+00,1.101700,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.244500,0.000000e+00,0.000000,0.000000e+00,676.530000,3.560700e+08,222.700000,93.500000,2.250000,95.430000,34.000000,24.000000,11.000000
25%,26.289250,2.512000e+05,4.063425,3.465000e+05,5.508300,6.057675e+04,55.708500,5.835000e+04,1.524100,9.528420e+05,9276.599600,1.238391e+08,1149.500000,1.662000e+09,365.982500,105.400000,3.000000,110.300000,41.250000,35.000000,14.000000
50%,32.522400,4.141000e+05,8.986750,5.455500e+05,11.492800,1.478535e+05,68.336900,9.200000e+04,2.747900,1.562724e+06,12262.300300,1.720943e+08,1355.810100,3.212320e+09,451.600000,115.600000,3.700000,118.330000,46.000000,38.000000,17.000000
75%,44.048600,6.444000e+05,23.034000,8.541750e+05,20.067175,3.018135e+05,87.773950,1.393000e+05,17.332400,2.369748e+06,14201.124750,2.135500e+08,1966.970000,3.919240e+09,580.987500,125.700000,4.750000,131.310000,60.000000,41.000000,26.750000
max,75.770000,1.482520e+07,58.520000,3.157200e+07,37.233700,5.316156e+06,117.956200,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,7.500000,150.480000,100.000000,100.000000,100.000000


In [382]:
data_transformed.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4356.000000,4356.000000,4356.000000,4104.000000,4104.000000,4104.000000
mean,37.109403,5.694220e+05,18.940323,7.199270e+05,15.265296,2.480761e+05,73.002794,1.209935e+05,11.782132,2.197581e+06,12590.821939,1.860655e+08,1624.011852,3.407804e+09,506.435424,118.977643,3.682266,124.289658,51.271107,38.847096,26.519336
std,13.029652,5.791911e+05,15.023725,8.261941e+05,7.693179,2.841019e+05,20.707893,1.040316e+05,12.599110,2.227285e+06,2530.510741,7.123267e+07,587.518266,1.349572e+09,119.183529,9.740415,1.124192,10.070194,14.764096,8.278884,20.730267
min,16.836000,2.450000e+04,2.986100,1.140000e+04,3.808200,2.385000e+03,33.211100,5.000000e+03,0.575400,6.559800e+04,6228.600100,1.824000e+05,676.530000,4.872200e+08,277.380000,102.006250,2.250000,96.700000,34.000000,24.000000,11.000000
25%,26.480875,2.926250e+05,6.716625,3.726000e+05,8.646075,7.874850e+04,56.387600,6.380000e+04,2.166825,1.180112e+06,11423.900400,1.465953e+08,1182.600000,2.566928e+09,407.732500,111.606395,2.992500,116.290000,41.314723,35.603488,14.046512
50%,34.172400,4.553000e+05,12.103150,5.636000e+05,14.227450,1.768170e+05,69.621600,9.615000e+04,3.743100,1.680761e+06,12813.750000,1.844498e+08,1400.035000,3.400680e+09,498.376000,119.850000,3.000000,123.420000,46.000000,38.043478,16.608696
75%,48.450775,6.820000e+05,33.404750,8.600250e+05,21.239800,3.264015e+05,92.728800,1.443250e+05,25.906150,2.493138e+06,14594.175050,2.212734e+08,2060.997500,4.066398e+09,605.208500,126.986379,4.250000,133.832500,59.965909,40.627907,25.662285
max,75.770000,1.482520e+07,58.520000,3.157200e+07,37.233700,4.790751e+06,117.956200,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,6.250000,150.480000,100.000000,100.000000,100.000000


In [383]:
data_transformed.head()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2003-01-02,31.6037,134900.0,3.1325,101400.0,5.1538,17982.0,65.4457,133000.0,0.7302,296784.0,6740.1001,76406700.0,909.03,1.229200e+09,305.898,102.00625,4.5,96.70,NaN,NaN,NaN
2003-01-03,31.3404,48500.0,3.1501,273600.0,5.1797,34761.0,65.3670,9300.0,0.7091,2904300.0,6772.7002,74780400.0,908.59,1.130800e+09,306.836,102.01250,4.5,97.08,NaN,NaN,NaN
2003-01-06,31.5452,232700.0,3.1361,161700.0,5.1971,64512.0,65.5529,99400.0,0.7036,541584.0,6837.2998,142266300.0,929.01,1.435900e+09,307.774,102.01875,4.5,97.34,NaN,NaN,NaN
2003-01-07,31.6037,842600.0,3.1501,945900.0,5.2317,366657.0,65.7675,54400.0,0.7004,685632.0,6802.7998,150351200.0,922.93,1.545200e+09,308.712,102.02500,4.5,97.42,NaN,NaN,NaN
2003-01-08,31.5452,423900.0,3.1325,301800.0,5.1624,179112.0,65.4814,227400.0,0.6759,1697196.0,6723.1001,145587400.0,909.93,1.467600e+09,309.650,102.03125,4.5,97.21,NaN,NaN,NaN


In [384]:
data_transformed.tail()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2019-10-14,72.805,437050.0,56.50,505550.0,35.025,501072.0,109.77,104600.0,39.505,1352527.0,16416.7998,206539800.0,2966.1499,2.557020e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-15,72.600,446100.0,56.59,627000.0,34.920,386557.0,109.30,107100.0,39.680,1290026.0,16418.4004,212333000.0,2995.6799,3.340740e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-16,71.620,716200.0,56.06,435900.0,34.700,805510.0,109.22,170900.0,39.580,1623424.0,16427.1992,165968300.0,2989.6899,3.222570e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-17,71.500,528200.0,56.23,700600.0,35.130,576685.0,109.28,120000.0,39.170,1868090.0,16426.3008,168397700.0,2997.9500,3.115960e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0
2019-10-18,71.220,284800.0,55.95,633400.0,34.840,458759.0,108.84,72500.0,39.020,1669638.0,16377.0996,166347800.0,2986.2000,3.264290e+09,432.33,136.8,3.95,117.75,80.0,34.0,64.0


In [385]:
data_transformed.to_csv('data_transformed.csv')

#### 3. Load the Data

In [386]:
def load_data(data_transformed_filename):
    '''
    save the transformed data to csv file
    '''
    data_loaded = pd.read_csv(data_transformed_filename, parse_dates=['date'], index_col='date')

    return data_loaded

In [387]:
data_loaded = load_data("data_transformed.csv")

In [388]:
data_loaded.head()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
date,,,,,,,,,,,,,,,,,,,,,
2003-01-02,31.6037,134900.0,3.1325,101400.0,5.1538,17982.0,65.4457,133000.0,0.7302,296784.0,6740.1001,76406700.0,909.03,1.229200e+09,305.898,102.00625,4.5,96.70,NaN,NaN,NaN
2003-01-03,31.3404,48500.0,3.1501,273600.0,5.1797,34761.0,65.3670,9300.0,0.7091,2904300.0,6772.7002,74780400.0,908.59,1.130800e+09,306.836,102.01250,4.5,97.08,NaN,NaN,NaN
2003-01-06,31.5452,232700.0,3.1361,161700.0,5.1971,64512.0,65.5529,99400.0,0.7036,541584.0,6837.2998,142266300.0,929.01,1.435900e+09,307.774,102.01875,4.5,97.34,NaN,NaN,NaN
2003-01-07,31.6037,842600.0,3.1501,945900.0,5.2317,366657.0,65.7675,54400.0,0.7004,685632.0,6802.7998,150351200.0,922.93,1.545200e+09,308.712,102.02500,4.5,97.42,NaN,NaN,NaN
2003-01-08,31.5452,423900.0,3.1325,301800.0,5.1624,179112.0,65.4814,227400.0,0.6759,1697196.0,6723.1001,145587400.0,909.93,1.467600e+09,309.650,102.03125,4.5,97.21,NaN,NaN,NaN


In [389]:
data_loaded.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4356 entries, 2003-01-02 to 2019-10-18
Data columns (total 21 columns):
loblaw_price           4356 non-null float64
loblaw_volume          4356 non-null float64
metro_price            4356 non-null float64
metro_volume           4356 non-null float64
empa_price             4356 non-null float64
empa_volume            4356 non-null float64
gwl_price              4356 non-null float64
gwl_volume             4356 non-null float64
atd_price              4356 non-null float64
atd_volume             4356 non-null float64
tsx_price              4356 non-null float64
tsx_volume             4356 non-null float64
sp500_price            4356 non-null float64
sp500_volume           4356 non-null float64
BCPI                   4356 non-null float64
CPI                    4356 non-null float64
bank_interest          4356 non-null float64
CEER                   4356 non-null float64
trend_grocery_store    4104 non-null float64
trend_loblaws       

In [390]:
data_loaded.describe()

,loblaw_price,loblaw_volume,metro_price,metro_volume,empa_price,empa_volume,gwl_price,gwl_volume,atd_price,atd_volume,tsx_price,tsx_volume,sp500_price,sp500_volume,BCPI,CPI,bank_interest,CEER,trend_grocery_store,trend_loblaws,trend_stock
count,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4.356000e+03,4356.000000,4356.000000,4356.000000,4356.000000,4104.000000,4104.000000,4104.000000
mean,37.109403,5.694220e+05,18.940323,7.199270e+05,15.265296,2.480761e+05,73.002794,1.209935e+05,11.782132,2.197581e+06,12590.821939,1.860655e+08,1624.011852,3.407804e+09,506.435424,118.977643,3.682266,124.289658,51.271107,38.847096,26.519336
std,13.029652,5.791911e+05,15.023725,8.261941e+05,7.693179,2.841019e+05,20.707893,1.040316e+05,12.599110,2.227285e+06,2530.510741,7.123267e+07,587.518266,1.349572e+09,119.183529,9.740415,1.124192,10.070194,14.764096,8.278884,20.730267
min,16.836000,2.450000e+04,2.986100,1.140000e+04,3.808200,2.385000e+03,33.211100,5.000000e+03,0.575400,6.559800e+04,6228.600100,1.824000e+05,676.530000,4.872200e+08,277.380000,102.006250,2.250000,96.700000,34.000000,24.000000,11.000000
25%,26.480875,2.926250e+05,6.716625,3.726000e+05,8.646075,7.874850e+04,56.387600,6.380000e+04,2.166825,1.180112e+06,11423.900400,1.465953e+08,1182.600000,2.566928e+09,407.732500,111.606395,2.992500,116.290000,41.314723,35.603488,14.046512
50%,34.172400,4.553000e+05,12.103150,5.636000e+05,14.227450,1.768170e+05,69.621600,9.615000e+04,3.743100,1.680761e+06,12813.750000,1.844498e+08,1400.035000,3.400680e+09,498.376000,119.850000,3.000000,123.420000,46.000000,38.043478,16.608696
75%,48.450775,6.820000e+05,33.404750,8.600250e+05,21.239800,3.264015e+05,92.728800,1.443250e+05,25.906150,2.493138e+06,14594.175050,2.212734e+08,2060.997500,4.066398e+09,605.208500,126.986379,4.250000,133.832500,59.965909,40.627907,25.662285
max,75.770000,1.482520e+07,58.520000,3.157200e+07,37.233700,4.790751e+06,117.956200,1.635000e+06,43.794600,6.585227e+07,16899.699200,8.588881e+08,3025.860100,1.145623e+10,911.940000,137.000000,6.250000,150.480000,100.000000,100.000000,100.000000
